In [1]:
import os
import math
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image
import time
import copy
from datetime import datetime

In [2]:
BATCH_SIZE = 1
LABELS = sorted([i[:-4] for i in os.listdir('icons')])
NUM_CLASSES = len(LABELS)

In [3]:
import numpy as np
import collections

class GaussianNoise(object):
    """
    Add gaussian noise to a numpy.ndarray (H x W x C)
    """
    def __init__(self, mean, sigma, random_state=np.random):
        self.sigma = sigma
        self.mean = mean
        self.random_state = random_state

    def __call__(self, image):
        row, col, ch = image.shape
        gauss = self.random_state.normal(self.mean, self.sigma, (row, col, ch))
        gauss = gauss.reshape(row, col, ch)
        image += torch.from_numpy(gauss).float()
        return image

In [4]:
class IconsDataset(Dataset):
    def __init__(self, directory, labels, transform=None):
        self.directory = directory
        self.files = os.listdir(directory)[:1000]
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        f = self.files[index]
        # image
        image = Image.open(os.path.join(self.directory, f))
        if self.transform:
            image = self.transform(image)
        
        # labels
        label = f.split('.')[0].split('_')[1]
        label = LABELS.index(label)
        return image, torch.tensor(label, dtype=torch.long)
    
    def input_size(self):
        return 100 * 100 * 3
        #return 224 * 224 * 3
    
    def __len__(self):
        return len(self.files)


def load_dataset():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        #GaussianNoise(0.01, 0.001),
    ])
    
    d = IconsDataset('images_single/', LABELS, transform=transform)
    size = len(d)
    train, validate = random_split(d, [int(size * 0.8), int(size * 0.2)])
    
    loader = DataLoader(train, batch_size=BATCH_SIZE)
    validation_loader = DataLoader(validate, batch_size=BATCH_SIZE)

    return d.input_size(), loader, validation_loader

In [5]:
class ModelConv1(nn.Module):
    def __init__(self, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()
        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv2d(3, conv_out_channels, kernel_size=kernel_size)
        conv_layer_output_size = int(100 - (kernel_size - 1))
        self.pooled_pixels = int(conv_layer_output_size / 2)
        self.h1 = nn.Linear(self.pooled_pixels * self.pooled_pixels  * conv_out_channels, linear_size)
        self.h2 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)

        x = x.view(-1, self.pooled_pixels * self.pooled_pixels * self.conv_out_channels)

        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h2(x)
        x = F.softmax(x, dim=1)
        return x

In [6]:
class ModelConv2(nn.Module):
    def __init__(self, input_size, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()

        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv2d(3, conv_out_channels, kernel_size=kernel_size)
        self.conv2 = nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)
        #self.conv3 = nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)

        #conv_layer_output_size = int(input_size - (kernel_size - 1))
        x = input_size
        x = x - (kernel_size - 1)
        x = int(x / 2)
        
        x = x - (kernel_size - 1)
        x = int(x / 2)
        
        #x = x - (kernel_size - 1)
        #x = int(x / 2)
        self.pooled_samples = x * x * conv_out_channels

        self.h1 = nn.Linear(self.pooled_samples, linear_size)
        self.h2 = nn.Linear(linear_size, linear_size)
        self.h3 = nn.Linear(linear_size, linear_size)
        self.h4 = nn.Linear(linear_size, linear_size)
        self.h9 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = F.max_pool2d(x, 2, 2)
        
        x = self.conv2(x)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = F.max_pool2d(x, 2, 2)
        
        #x = self.conv3(x)
        #x = F.relu(x)
        #x = F.max_pool2d(x, 2, 2)

        x = x.view(BATCH_SIZE, self.pooled_samples)
        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h4(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [7]:
class ModelConv2x3(nn.Module):
    def __init__(self, input_size, conv_out_channels=5, linear_size=50):
        super().__init__()
        x = int(input_size / 2 / 2 / 2)
        self.pooled_samples = x * x * conv_out_channels
        self.conv1 = nn.Conv2d(3, conv_out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=3, padding=1)

        self.h1 = nn.Linear(self.pooled_samples, linear_size)
        self.h2 = nn.Linear(linear_size, linear_size)
        self.h3 = nn.Linear(linear_size, linear_size)
        self.h4 = nn.Linear(linear_size, linear_size)
        self.h9 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)

        x = self.conv5(x)
        x = F.relu(x)
        x = self.conv5(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)


        x = x.view(BATCH_SIZE, self.pooled_samples)
        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h4(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [8]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    f = open(f'{datetime.now().isoformat()}.txt', 'w', buffering=1)


    for epoch in range(num_epochs):
        train_loss = None
        train_accuracy = None

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            if phase == 'train':
                train_loss = epoch_loss
                train_accuracy = epoch_acc
            else:
                stats = f'Epoch: {epoch}, TL: {train_loss:.5f}, VL: {epoch_loss:.5f}, TA: {train_accuracy:.5f}, VA: {epoch_acc:.5f}'
                print(stats)
                f.write(f'{stats}\n')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [9]:
#input_size, loader, validation_loader = load_dataset()
# model = ModelConv1(kernel_size=5, conv_out_channels=20, linear_size=500).cuda()
#model = ModelConv2(input_size=100, kernel_size=5, conv_out_channels=20, linear_size=500).cuda()
#learn(model, loader, validation_loader, epochs=10000, learning_rate=0.0001)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model = ModelConv1(kernel_size=5, conv_out_channels=20, linear_size=500).cuda()
#model = ModelConv2(input_size=100, kernel_size=5, conv_out_channels=20, linear_size=500).cuda()
model = ModelConv2x3(input_size=224, conv_out_channels=64, linear_size=500).cuda()

input_size, loader, validation_loader = load_dataset()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(params=model.parameters(), lr=0.0001)
train_model(model, {'train': loader, 'val': validation_loader}, criterion, optimizer, device, num_epochs=1000)

TypeError: __init__() missing 1 required positional argument: 'input_size'